In [7]:
def foda_classif(diret,key,target):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2
    from sklearn.preprocessing import LabelEncoder
    from sklearn import preprocessing
    from sklearn.ensemble import RandomForestClassifier
    
    data=pd.read_csv(diret)

    if data[target[0]].dtypes=='object':
        le = LabelEncoder()
        data[target] = le.fit_transform(data[target])
        print('-'*120)
        print(' ')
        print('Ajustando formato da variável Alvo')
        print(' ')
        print('Originalmente os valores eram: ')
        print(list(le.classes_))
        print(' ')
        print('E agora são: ')
        print(list(le.transform(le.classes_)))


    if data.shape[0]>50000:
        print('-'*115)
        print(' ')
        print('Dado o tamanho da base estamos fazendo uma amostra de tamanho:')
        data=data.sample(n=50000, random_state=1)
        print(data.shape)
        print(' ')

    print('-'*115)
    print(' ')
    print('Informações sobre os dados:')
    print(' ')
    print(data.info())
    print(' ')
    print('-'*115)
    c = (data.dtypes == 'object')
    categorical_data = list(c[c].index)

    n = (data.dtypes == 'int64')|(data.dtypes == 'float64')
    numerical_data = list(n[n].index)

    out=target+key

    i=0
    while i<len(out):
        if data[out[i]].dtypes=='object':
            categorical_data.remove(out[i])
        else:
            numerical_data.remove(out[i])
        i=i+1
    print(' ')
    print('Variáveis Categóricas: ')
    print(categorical_data)
    print(' ')
    print('Variáveis Numéricas: ')
    print(numerical_data)
    print(' ')
    i=0
    categorica=[]
    num=[]
    print('-'*115)
    print(' ')
    print('Análises das Variáveis Categóricas')
    print(' ')
    print('-'*115)
    print(' ')
    while i<len(categorical_data):
        print("Variável: "+categorical_data[i])
        if len(data[categorical_data[i]].unique())<=20:
            categorica.append(categorical_data[i])
            print(data[categorical_data[i]].fillna('NaN').unique())
            print(' ')
            print(data[[categorical_data[i],target[0]]].fillna('NaN').groupby(categorical_data[i]).agg(['count','sum','mean']))
            print(' ')
            fig, [axis0,axis1] = plt.subplots(1,2,figsize=(10,5))
            data[categorical_data[i]].value_counts().plot.pie(autopct='%1.1f%%',ax=axis0)
            ax = sns.countplot(x = categorical_data[i],hue = target[0],data=data.fillna('NaN'))
            plt.show()
            print('-'*115)
        else:
            print("Variável com {} valores distintos, recomendamos algum tratamento nesta informação para redução de classes.".format(len(data[categorical_data[i]].unique())))
            print(' ')
            print('-'*115)
        print(' ')    
        i=i+1
    i=0  
    print('Análises das Variáveis Numéricas')
    print(' ')
    print('-'*115)
    print(' ')
    ordinal=[]
    continua=[]
    while i<len(numerical_data):
        print("Variável: "+numerical_data[i])
        num.append(numerical_data[i])
        if len(data[numerical_data[i]].unique())<=6 and data[numerical_data[i]].dtypes == 'int64':
            ordinal.append(numerical_data[i])
            print(data[numerical_data[i]].unique())
            print(' ')
            print(data[[numerical_data[i],target[0]]].groupby(numerical_data[i]).agg(['count','sum','mean']))
            print(' ')
            fig, [axis0,axis1] = plt.subplots(1,2,figsize=(10,5))
            data[numerical_data[i]].value_counts().plot.pie(autopct='%1.1f%%',ax=axis0)      
            ax = sns.countplot(x = numerical_data[i],hue = target[0],data=data.fillna(data.mean()))
            plt.show()
            ax =  sns.violinplot(x=target[0], y=numerical_data[i], data=data.fillna(data.mean()))
            plt.show()
            print('-'*115)
        else:
            continua.append(numerical_data[i])
            xis=data[numerical_data[i]].fillna(data[numerical_data[i]].mean())
            ax = sns.distplot(xis)
            plt.show()
            print(' ')
            print('Média: {}'.format(str(data[numerical_data[i]].fillna(data[numerical_data[i]].mean()).mean())))
            print('DesvioPadrão: {}'.format(str(data[numerical_data[i]].fillna(data[numerical_data[i]].mean()).std())))
            print(' ')
            t=0
            x={}
            while t < len(data[target[0]].unique()):
                x[t]=data[data[target[0]]==data[target[0]].unique()[t]]
                sns.distplot(a=x[t][numerical_data[i]].fillna(x[t][numerical_data[i]].mean()), label=str(data[target[0]].unique()[t]), kde=True)
                t=t+1
            plt.legend()
            plt.show()
            t=0
            while t < len(data[target[0]].unique()):
                print('Média (Alvo = {}): {}'.format(str(data[target[0]].unique()[t]),str(x[t][numerical_data[i]].fillna(x[t][numerical_data[i]].mean()).mean())))
                print('DesvioPadrão (Alvo = {}): {}'.format(str(data[target[0]].unique()[t]),str(x[t][numerical_data[i]].fillna(x[t][numerical_data[i]].mean()).std())))
                print(' ')
                t=t+1
            print(' ')
            ax =  sns.violinplot(x=target[0], y=numerical_data[i], data=data.fillna(data.mean()))
            plt.show()
            print('-'*115)
            print(' ')
            plt.show()
        i=i+1
    print('Análise das Variáveis Conjuntas')
    print(' ')
    print('-'*115)
    print(' ')
    data=pd.read_csv(diret)
    if data.shape[0]>10000:
        data=data.sample(n=10000, random_state=1)
    print('Matriz de Correlação')
    fig = plt.figure(figsize=(20,20))
    sns.heatmap(data[continua].corr(), annot=True,fmt=".2f")
    plt.show()
    print(' ')
    print('-'*115)
    print(' ')
    g = sns.PairGrid(data[continua].fillna(data[continua].mean()))
    g = g.map_diag(plt.hist, edgecolor="w")
    g = g.map_offdiag(plt.scatter, edgecolor="w", s=40)
    plt.show()
    print(' ')
    print('-'*115)
    cont=continua+target
    print(' ')
    g = sns.pairplot(data[cont].fillna(data[cont].mean()),vars=cont[0:-1], hue=cont[-1])
    g = g.map_diag(plt.hist, edgecolor="w")
    g = g.map_offdiag(plt.scatter, edgecolor="w", s=40)
    plt.show()
    print(' ')
    print('-'*115)
    print(' ')
    print('Importância das Variáveis')
    print(' ')
    print('-'*115)
    print(' ')
    vari=categorica+numerical_data
    data[categorica].head()
    label_data = data.copy()
    label_encoder = LabelEncoder()
    col_categorica=[]
    for col in categorica:
        label_data[col] = label_encoder.fit_transform(data[col].fillna('NaN'))
    rforest_checker = RandomForestClassifier(random_state = 0)
    rforest_checker.fit(label_data[categorica+numerical_data].fillna(label_data[categorica+numerical_data].mean()), data[target])
    importances_df = pd.DataFrame(rforest_checker.feature_importances_, columns=['Feature_Importance'],index=label_data[categorica+numerical_data].fillna(label_data[categorica+numerical_data].mean()).columns)
    importances_df.sort_values(by=['Feature_Importance'], ascending=False, inplace=True)
    fig = plt.figure(figsize=(20,20))
    sns.barplot(x=importances_df.columns[0],y=importances_df.index,data=importances_df)
    plt.show()
    data=pd.read_csv(diret)

In [8]:
diretorio='train.csv'
chave=['PassengerId','Name']
alvo=['Survived']